## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. 
Нужно понять, влияет ли семейное положение и количество детей клиента на погашение кредита в срок. 
Входные данные от заказчика — статистика о платёжеспособности клиентов.

Результаты исследования будут использованы для построения модели **кредитного скоринга**.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [88]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

In [89]:
data.info() #просматриваю общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [90]:
data.head() #знакомлюсь с данными

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Найдены пропуски в столбцах days_employed и total_income

### Шаг 2. Предобработка данных

### Обработка пропусков

In [91]:
print('Количество пропусков',data.isnull().sum())
total_income_NaN = data[data['total_income'].isnull()]
print('')
print('Количество записей с пустым доходом по типу занятости', total_income_NaN.groupby('income_type')['income_type'].count()) 
print('')
print('Медиана дохода по типу занятости', data.groupby('income_type')['total_income'].median())
#считаю средний доход для людей с определенным типом занятости

data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda group: group.fillna(group.median()))
#заполним пропуск дохода медианным значением по типу занятости
print('')
print('Количество пропусков в доходе после заполнения медианным значением по типу занятости',data['total_income'].isnull().sum())
#проверяем остались ли пропуски
                                                

Количество пропусков children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество записей с пустым доходом по типу занятости income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

Медиана дохода по типу занятости income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Количество пропусков в доходе после заполнения медианным значением по типу занятости 0


### Вывод

В таблице есть пропуск дохода и стажа (по 2174 строки).

Исследуем состав людей с пустым доходом, сгруппированные по типу дохода:
income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105

По таблице можно увидеть, что все люди с пропущенными данными - работающие. То есть их стаж и доход не должен быть пустым. 

Что делать с пропусками:
- Пропуск в days_employed не критичен для расчетов. Его оставим без изменений
- Пропуск в total_income заполним медианным значением дохода по людям с тем же типом дохода (income_type).





### Замена типа данных

In [92]:
#print(data.head(3))
print('Пустые значения в столбцах: ')
print(data.isna().sum())
data = data.fillna(0) #Заполняем пустые значения, иначе не получится конвертировать тип данных в int64
data['days_employed'] = data['days_employed'].astype('int64') #Меняем тип float на int64 в стаже
data['total_income'] = data['total_income'].astype('int64') #Меняем тип float на int64 в дохоже



Пустые значения в столбцах: 
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


In [93]:
#Проверяем, что тип у total_income и days_employed теперь int64:

In [94]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [95]:
data['days_employed'] = data['days_employed'].abs() 
#Также меняем отрицательные значения в стаже на положительные по модулю. Это пригодится в вычислениях.

### Вывод

Столбцы, которые по смыслу являются целочисленными, а по факту - имеют тип float:
- days_employed
- total_income

Поменяем для них тип с float на int64 методом astype, который позволяет делать приведение к целочисленному типу.

### Обработка дубликатов

In [96]:
print ('Дубликатов в таблице:', data.duplicated().sum())
#так как в таблице были строки с разным регистром, то приведем все к нижнему регистру
print(data.columns)
data['education'] = data['education'].str.lower()
data['gender'] = data['gender'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
print('')


Дубликатов в таблице: 54
Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')



In [97]:

print('проверим, правда ли привели строковые данные к нижнему регистру')
print(data.head(5))

print('')
print ('Теперь дубликатов в таблице:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print('')
print ('Проверяем, что не осталось дубликатов после их удаления:', data.duplicated().sum())


проверим, правда ли привели строковые данные к нижнему регистру
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      f   сотрудник     0        253875   
1   женат / замужем                 0      f   сотрудник     0        112080   
2   женат / замужем                 0      m   сотрудник     0        145885   
3   женат / замужем                 0      m   сотрудник     0        267628   
4  гражданский брак                 1      f   пенсионер     0        158616   

                      purpose  
0               покупка жилья  


### Вывод

Исходное количество дубликатов - 54. 
Считаем их количество через поиск полных дубликатов строк методом .duplicated().sum().
Приводим к нижнему регистру строковые значения в таблице - и дубликатов становится 71.
Удаляем дубликаты методом .drop_duplicates().reset_index(drop=True).
Проверяем, что они удалились.

Вероятные причины появления дубликатов:
- либо повторные выгрузки с пересечением данных
- либо ошибки в обработке событий
- либо сбои в логировании

### Лемматизация

In [98]:
# pymystem3 импортируется так:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [99]:
#Лемматизирую и считаю количество лемм в списке уникальных целей кредитов для наглядности
list_purpose = data['purpose'].unique() #Выделяю список уникальных целей кредита
#print(list_purpose)
lemmas = [] 
print('Леммы в списке уникальных целей кредитов:')
for row in list_purpose: 
    row_text = str(row)
    lemmas = m.lemmatize(row_text)
    print(Counter(lemmas))

Леммы в списке уникальных целей кредитов:
Counter({'покупка': 1, ' ': 1, 'жилье': 1, '\n': 1})
Counter({'приобретение': 1, ' ': 1, 'автомобиль': 1, '\n': 1})
Counter({'дополнительный': 1, ' ': 1, 'образование': 1, '\n': 1})
Counter({'сыграть': 1, ' ': 1, 'свадьба': 1, '\n': 1})
Counter({' ': 2, 'операция': 1, 'с': 1, 'жилье': 1, '\n': 1})
Counter({'образование': 1, '\n': 1})
Counter({' ': 2, 'на': 1, 'проведение': 1, 'свадьба': 1, '\n': 1})
Counter({' ': 3, 'покупка': 1, 'жилье': 1, 'для': 1, 'семья': 1, '\n': 1})
Counter({'покупка': 1, ' ': 1, 'недвижимость': 1, '\n': 1})
Counter({' ': 2, 'покупка': 1, 'коммерческий': 1, 'недвижимость': 1, '\n': 1})
Counter({' ': 2, 'покупка': 1, 'жилой': 1, 'недвижимость': 1, '\n': 1})
Counter({' ': 2, 'строительство': 1, 'собственный': 1, 'недвижимость': 1, '\n': 1})
Counter({'недвижимость': 1, '\n': 1})
Counter({'строительство': 1, ' ': 1, 'недвижимость': 1, '\n': 1})
Counter({' ': 3, 'на': 1, 'покупка': 1, 'подержать': 1, 'автомобиль': 1, '\n': 1}

In [100]:
lemmas = []    
def purpose_group(purpose): #Делаем функцию, которая вернет категорию кредита по цели кредита
    row_text = str(purpose)
    lemmas = m.lemmatize(row_text)
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'операции с недвижимостью'
    if 'автомобиль' in lemmas:
        return 'покупка авто'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'обучение'
    else: 
        return 'нет группы цели кредита'

data['purpose_group'] = data['purpose'].apply(purpose_group)

In [101]:
#Всего заявок по цели кредита:

In [102]:
data['purpose_group'].value_counts()

операции с недвижимостью    10811
покупка авто                 4306
обучение                     4013
свадьба                      2324
Name: purpose_group, dtype: int64

### Вывод

Лемматизация проведена по уникальным значениям столбца 'purpose', который содержит цель кредита.
Выделены леммы по каждой цели кредита.

В целом, можно увидеть что в списке лемм есть 4 группы целей кредита:
- авто
- недвижимость
- свадьба
- обучение.

Создан отдельный столбец 'purpose_group'. 
Каждой строке dataframe присвоена группа цели кредита в зависимости от лемм, найденных в цели кредита.

Получено такое количество заявок по группе цели кредита:
операции с недвижимостью    10811
покупка авто                 4306
обучение                     4013
свадьба                      2324.

### Категоризация данных

In [103]:
#Посмотрим обзорно на таблицу

In [104]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.539946,60355.050667,43.271231,0.817097,0.973898,0.081150,1.653196e+05
std,1.383444,133531.412724,12.570822,0.548674,1.421567,0.273072,9.818730e+04
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,622.000000,33.000000,1.000000,0.000000,0.000000,1.076230e+05
50%,0.000000,1821.500000,42.000000,1.000000,0.000000,0.000000,1.425940e+05
75%,1.000000,4797.000000,53.000000,1.000000,1.000000,0.000000,1.958202e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [105]:
#Выделяем категории уровням дохода

In [106]:
total_income_min = data['total_income'].min()
total_income_max = data['total_income'].max()
total_income_mean = data['total_income'].mean()

def income_level(total_income): #Считаю уровень дохода
    if total_income <= total_income_min:
        return('Минимальный доход')
    if (total_income > total_income_min) & (total_income <= total_income_mean):
        return('Доход меньше или равен среднему')
    if (total_income > total_income_mean) & (total_income <= total_income_max):
        return('Доход больше среднего')
    else: 
        return('Ух ты, больше максимума')

data['income_level'] = data['total_income'].apply(income_level)

In [107]:
#Выделяем категории количеству детей

In [108]:
def children_category(children): 
    if children <= 0:
        return('Нет детей')
    if (children >= 1 ) & (children <= 2):
        return('1-2 ребенка')
    if (children >= 3):
        return('3 детей и более')
    else: 
        return('Ошибка вычисления')

data['children_category'] = data['children'].apply(children_category)
#print(data.sample())

### Вывод

Для упрощения построения зависимостей и упрощения выводов можно выделить категории уровня дохода:
- минимальный
- меньше или равен среднему
- больше среднего
и категории наличия детей:
- нет детей
- 1-2 ребенка
- 3 и более детей

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [109]:
print('Зависимость возврата кредита в срок и наличия детей')
data_pivot_children = data.pivot_table(index='children_category', values='debt').sort_values(by='debt', ascending=True)

Зависимость возврата кредита в срок и наличия детей


In [110]:
data_pivot_children

,debt
children_category,
Нет детей,0.075258
3 детей и более,0.085526
1-2 ребенка,0.093003


### Вывод

Есть ли зависимость между наличием детей и возвратом кредита в срок?
1. Да. Можно предположить, что заемщики без детей наиболее стабильны с выплатами кредита в срок. Заемщики с количеством детей 1 или 2 - наименее стабильны в выплатах. Вероятно, они еще не привыкли распоряжаться бюджетом при наличии нового дестабилизрующего фактора - ребенка
- Наименьшее количество невозвратов в срок у заемщиков без детей. 
- Немного больше невозвратов у заемщиков с 3 и более детьми.
- Еще немного больше возвратов у заемщиков с 1-2 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [111]:
print('Зависимость возврата кредита в срок и семейного положения')
data_pivot_family_status = data.pivot_table(index='family_status', values='debt').sort_values(by='debt', ascending=True)

Зависимость возврата кредита в срок и семейного положения


In [112]:
data_pivot_family_status

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
не женат / не замужем,0.097509


### Вывод

Есть ли зависимость между семейным положением и возвратом кредита в срок?
1. Да. Меньше всего невозврата кредита в срок у вдовцов. Находящиеся в браке или разведенныее - следующая по надежности группа. И наименее надежными являются не состоящие в браке.
- Наиболее стабильны в возврате кредита в срок вдовы/вдовцы.
- Чуть менее стабильны в возврате кредита в срок женатые / замужние и те кто в разводе.
- Наименее стабильными в возврате кредита в срок являются состоящие в гражданском браке или не женатые/не замужние.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [113]:
print('Зависимость возврата кредита в срок и уровня дохода')
data_pivot_income = data.pivot_table(index='income_level', values='debt').sort_values(by='debt', ascending=True)

Зависимость возврата кредита в срок и уровня дохода


In [114]:
data_pivot_income

,debt
income_level,
Доход больше среднего,0.076368
Доход меньше или равен среднему,0.084023
Минимальный доход,1.000000


### Вывод

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
1. Да, есть зависимость. Чем выше уровень дохода, тем менее вероятен невозврат кредита

- Как разные цели кредита влияют на его возврат в срок?

In [115]:
print('Зависимость возврата кредита в срок и цели кредита')
data_pivot_purpose_group = data.pivot_table(index='purpose_group', values='debt').sort_values(by='debt', ascending=True)


Зависимость возврата кредита в срок и цели кредита


In [116]:
data_pivot_purpose_group

,debt
purpose_group,
операции с недвижимостью,0.072334
свадьба,0.080034
обучение,0.092200
покупка авто,0.093590


### Вывод

- Более надежными для возврата кредита в срок являются цели операции с недвижимостью и свадьба
- Менее надежными для возврата кредита в срок являются цели обучение и покупка авто.

### Шаг 4. Общий вывод

Есть факторы, влияющие на возврат кредита в срок.
Среди них можно выделить:
- Уровень дохода. Чем он выше, тем лучше возвращают кредит в срок
- Цель кредита. Недвижимость и свадьба - самые надежные цели кредита. Авто и обучение - менее надежные.
- Наличие детей. Отсутствие детей либо количество детей более 3 - показатели более надежного заемщика. При наличии 1-2 детей выплаты менее стабильны. Вероятно, заемщики недостаточно умеют управлять бюджетом, когда детей немного.
- Семейное положение. Люди, находящиеся в браке или уже побывавшие в нем, более надежные заемщики. Неженатые / незамужние или находящиеся в гражданском браке - менее надежные.


### Чек-лист готовности проекта

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.